In [1]:
import os
import json
import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session
import pandas as pd

In [2]:
sess = Session()

role = get_execution_role()

%store -r dummyModel_data

# Load in trained model from s3 bucket
try:
    dummyModel_data # loads in model from kernel
    
except NameError:
    import json

    # copy a pretrained model from our S3 bucket
    s3 = boto3.client("s3")
    bucket = sess.default_bucket()
    key = "actualModel/pytorch-training-2023-03-23-00-43-56-842/model.tar.gz"
    s3.download_file(bucket, key, "model.tar.gz")

    # upload to default bucket
    dummyModel_data = sess.upload_data(
        path="model.tar.gz", bucket=sess.default_bucket(), key_prefix="model/pytorch"
    )
    
print(dummyModel_data)

no stored variable or alias dummyModel_data
s3://sagemaker-us-east-1-472925017889/model/pytorch/model.tar.gz


In [25]:
model = PyTorchModel(
    entry_point="inference.py",
    source_dir="code",
    role=role,
    model_data=dummyModel_data,
    framework_version="1.5.0",
    py_version="py3",
)

In [ ]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

# Deploy the model
# set local_mode to False if you want to deploy on a remote

local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.m4.xlarge"
    
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer()
)

print()
print(predictor)

--------

In [ ]:
dummy_data = {"UserID": "5", "Dining Hall Preference": ["north", "northwest"], "Allergens": [], "Dietary Restrictions": [], "Meal": "Dinner"}
res = predictor.predict({"inputs" : dummy_data})
print(res)

In [24]:
# DELETING THE ENDPOINT

if not local_mode:
    predictor.delete_endpoint()
else:
    os.system("docker container ls | grep 8080 | awk '{print $1}' | xargs docker container rm -f")